# Training Model
1. Keras on Tensorflow
2. hyper-parameter search
3. vanila-Unet: pull the hyper parameter setting out from the function of model
4. date: 01312020


In [1]:
import os, sys
import cv2
import numpy as np
import uuid
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from imutils import paths
import itertools
import json
from pprint import pprint



import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from core.imageprep import dir_checker, random_crop, crop_generator, random_crop_batch
from core.models import UNet, UNet_hp, vanilla_unet, vanilla_unet_nodrop
from core.metrics import iou_coef, dice_coef

from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorboard.plugins.hparams import api as hp
from datetime import datetime

from IPython import get_ipython
# %load_ext autoreload
get_ipython().run_line_magic('load_ext', 'autoreload')
# %autoreload 2
get_ipython().run_line_magic('autoreload', '2')
# %load_ext tensorboard
get_ipython().run_line_magic('load_ext', 'tensorboard')

In [2]:
from packaging import version
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [3]:
sys.executable

'C:\\Users\\wucci_admin\\Anaconda3\\envs\\tfdl02\\python.exe'

 ## Load Training Dataset

In [4]:
# load image
print("Load Images...")
# on mac
# path = "/Volumes/LaCie_DataStorage/PerlmutterData/"

# on Window PC 
path = os.path.join('D:', 'PerlmutterData')

# input set
crop_input_set = '2020_01_23_09_51_20x'
# crop_input_set = '2020_01_30_11_24_1x' # small training set

imginput = os.path.join('dl_seg_project_raw', 'data_crop', crop_input_set,)
imgpath = os.path.join(path, imginput)

print(imgpath)

img_dir = os.path.join(imgpath, 'images')
label_dir = os.path.join(imgpath, 'labels')
print(img_dir)
print(label_dir)

# check if the output folder exist. If not, create a folder
dir_checker('logs', path)
path_logs = os.path.join(path, 'logs')
dir_checker('fit', path_logs)
dir_checker('model', path_logs)
dir_checker('pars', path_logs)

Load Images...
D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x
D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x\images
D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x\labels
logs exists in D:PerlmutterData
fit exists in D:PerlmutterData\logs
model exists in D:PerlmutterData\logs
pars exists in D:PerlmutterData\logs


 Print the first file name.

In [5]:
imgpath_all = list(paths.list_images(img_dir))
print(imgpath_all[0])

D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x\images\nucleus\0001.tif


 ## Create Image Datagenerator
 1. create only one datagen
 2. specify valiation split in datagen argument
 3. add split data when calling `datagen.flow_from_directory`

In [6]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
date =  datetime.now().strftime("%Y_%m_%d")
seed = 103
batch_size = 16
epoch = 20
validation_steps = 20
validation_split = 0.1
training_sample_size = len(imgpath_all)
IMG_HEIGHT = None
IMG_WIDTH = None
classes = ['cell_membrane', 'nucleus', 'autophagosome']
inputclass = [classes[1]]
learning_rate = 1e-4
loss = "binary_crossentropy"
metrics = ['accuracy', iou_coef, dice_coef]

metrics_name = []
for f in metrics:
    if callable(f):
        metrics_name.append(f.__name__)
    else:
        metrics_name.append(f)

In [7]:
print(training_sample_size)

6620


In [8]:
# create arguments for data generator
data_gen_img_args = dict(
                # featurewise_center = True,
                # featurewise_std_normalization = True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                rescale=1.0/255.0,
                )

data_gen_label_args = dict(
                # featurewise_center=True,
                # featurewise_std_normalization=True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                # rescale=1.0/255.0,
                rescale=1.0/255.0,
                )

# create parameter
pars = dict(
                # basic information
                timestamp = timestamp,
                date = date,
                seed = seed,
                batch_size = batch_size,
                
                # Data generator
                crop_input_set = crop_input_set,
                validation_steps = validation_steps,
                validation_split = validation_split,
                training_sample_size = training_sample_size,
                
                # training class
                classes = classes,
                inputclass = inputclass,
    
                # add datagen args
                data_gen_img_args = data_gen_img_args,
                data_gen_label_args = data_gen_label_args,
                
                # Build model
                IMG_HEIGHT = IMG_HEIGHT,
                IMG_WIDTH = IMG_WIDTH,
                epoch = epoch, 
                loss = loss,
                metrics_name = metrics_name,
                learning_rate = learning_rate,
                )

# save parameter
path_pars = os.path.join(path_logs, 'pars')
dir_checker(inputclass[0], path_pars)
dir_checker(date, os.path.join(path_pars, inputclass[0]))

pprint(pars)

nucleus exists in D:PerlmutterData\logs\pars
2020_02_06 exists in D:PerlmutterData\logs\pars\nucleus
{'IMG_HEIGHT': None,
 'IMG_WIDTH': None,
 'batch_size': 16,
 'classes': ['cell_membrane', 'nucleus', 'autophagosome'],
 'crop_input_set': '2020_01_23_09_51_20x',
 'data_gen_img_args': {'height_shift_range': 0.1,
                       'horizontal_flip': True,
                       'rescale': 0.00392156862745098,
                       'rotation_range': 90.0,
                       'shear_range': 0.07,
                       'validation_split': 0.1,
                       'vertical_flip': True,
                       'width_shift_range': 0.1,
                       'zoom_range': 0.2},
 'data_gen_label_args': {'height_shift_range': 0.1,
                         'horizontal_flip': True,
                         'rescale': 0.00392156862745098,
                         'rotation_range': 90.0,
                         'shear_range': 0.07,
                         'validation_split': 0.1,
   

In [9]:
par_file_dir = os.path.join(path_pars, inputclass[0], date, 'pars_' + timestamp + '.json')
print(par_file_dir)

with open(par_file_dir, 'w') as outfile:
    json.dump(pars, outfile, indent=4)


D:PerlmutterData\logs\pars\nucleus\2020_02_06\pars_2020_02_06_16_39.json


In [10]:
# create generator
image_datagen = ImageDataGenerator(**data_gen_img_args)
label_datagen = ImageDataGenerator(**data_gen_label_args)

In [11]:
# load images into generator
train_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training', # <- define subset as 'training'
    seed=seed)

train_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training',
    seed=seed)

valid_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation', # <- define subset as 'validation'
    seed=seed)

valid_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation',
    seed=seed)

Found 5958 images belonging to 1 classes.
Found 5958 images belonging to 1 classes.
Found 662 images belonging to 1 classes.
Found 662 images belonging to 1 classes.


In [12]:

# merge image and label generator
def combine_generator(gen1, gen2):
    while True:
        yield(gen1.next(), gen2.next()) 
train_generator = combine_generator(train_image_generator, train_label_generator)
valid_generator = combine_generator(valid_image_generator, valid_label_generator)


In [13]:
'''
train_generator = zip(train_image_generator, train_label_generator)
valid_generator = zip(valid_image_generator, valid_label_generator)
'''

'\ntrain_generator = zip(train_image_generator, train_label_generator)\nvalid_generator = zip(valid_image_generator, valid_label_generator)\n'

 ## Training

In [14]:
print("Start training...")

Start training...


 ### Define Callbacks

 ### Setup the model

In [15]:
# calculate steps_per_epoch
steps_per_epoch = training_sample_size * (1 - validation_split) // batch_size + 1
print("Steps per epoch: {}".format(steps_per_epoch))

Steps per epoch: 373.0


## Hyper-parameters grid search

In [16]:
# Hyperparameter
# Create a .v2 file for saving hyperparameter and evaluation
# so we can see the results on tensorboard
hparamtuning_dir = os.path.join(path_logs, 'fit', inputclass[0], date, timestamp)

HP_LEARNINGRATE = hp.HParam('learning_rate', hp.Discrete([0.1, 1e-2, 1e-3, 1e-4]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.5, 0.7]))
HP_LAYERS = hp.HParam('layers', hp.Discrete([4, 5]))

# hparams_list = [HP_DROPOUT, HP_LAYERS]

with tf.summary.create_file_writer(hparamtuning_dir).as_default():
    hp.hparams_config(
        hparams=[HP_LEARNINGRATE, HP_DROPOUT, HP_LAYERS],
        metrics=[hp.Metric('accuracy', display_name='Accuracy'), 
                 hp.Metric('iou_coef', display_name='IoU_Coef'), # create container for customized metrics
                 hp.Metric('dice_coef', display_name='Dice_Coef')], # the same
  )


In [17]:
def run(run_name, hparamtuning_dir, hparams):
    
    # checkpoint
    modelfilename = 'model_' + timestamp + '.h5'
    dir_checker(run_name, hparamtuning_dir)
    dir_checker('model', os.path.join(hparamtuning_dir, run_name))
    
    modelfile_path = os.path.join(hparamtuning_dir, run_name, 'model', modelfilename)
    checkpointer = ModelCheckpoint(filepath = modelfile_path, 
                                   monitor = 'val_accuracy', 
                                   mode = 'max', 
                                   verbose = 1, 
                                   save_best_only = True, 
                                  )

    # early stopping 
    early_stopping = EarlyStopping(monitor='val_loss',
                               patience=8,
                               verbose=1,
                               min_delta=1e-4)

    # learning rate adjustment
    reduceLR = ReduceLROnPlateau(monitor='val_loss',
                        factor=0.1,
                        patience=4,
                        verbose=1,
                        min_delta=1e-3,
                        cooldown = 2,
                        )

    # tensorboard ----------------------------------------------
    
    # file_writer = create_file_writer(os.path.join(path_logs, 'fit', inputclass[0], date, timestamp, "metrics"))
    # file_writer.set_as_default()

    metrics = ['accuracy', iou_coef, dice_coef]
    
    tensorboard_callback = TensorBoard(log_dir = os.path.join(hparamtuning_dir, run_name), 
                                       profile_batch = 0, 
                                       update_freq= 30,
                                       histogram_freq = 1
                                       )

    # compile callbacks
    # callbacks = [checkpointer, tensorboard_callback, early_stopping, reduceLR]
    callbacks = [checkpointer, reduceLR, tensorboard_callback]
    
    hparamtuning_runname_dir = os.path.join(hparamtuning_dir, run_name)
    
    
    with tf.summary.create_file_writer(hparamtuning_runname_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial

        # prepare the model -----------------------------------
        
        # load hyper-parameter
        learning_rate = float(hparams[HP_LEARNINGRATE])
        print('learning rate: {}'.format(learning_rate))
        
        dropout = float(hparams[HP_DROPOUT])
        print('dropout: {}'.format(dropout))
        
        num_layers = int(hparams[HP_LAYERS])
        print('num layers: {}'.format(num_layers))
        
        unetmodel = vanilla_unet_nodrop(
                            shape = (IMG_HEIGHT, IMG_WIDTH), 
                            dropout = dropout, 
                            num_layers = num_layers, 
                            lr = learning_rate, 
                            loss = loss,
                            metrics = metrics,
                            summary = False,
                           )
        
        # load model ------------------------------------------
        
        '''
        # load weight
        path_model = os.path.join('D:', 'PerlmutterData', 'logs', 'fit', 
                                'nucleus', 
                                '2020_02_05',
                                '2020_02_05_16_29',
                                'run-2',
                                'model',
                                'model_2020_02_05_16_29.h5',)
        
        unetmodel.load_weights(path_model)
        '''
        
        
        # train the model -------------------------------------
        unetmodel.fit_generator(
                            generator = train_generator, 
                            validation_data = valid_generator,
                            validation_steps = validation_steps,
                            steps_per_epoch = steps_per_epoch,
                            epochs = epoch,  
                            callbacks = callbacks,
                            verbose = 1, 
                            )
    
        _, accuracy, iou, dice,  = unetmodel.evaluate_generator(valid_generator, steps = 50, verbose=1)
        tf.summary.scalar('accuracy', accuracy, step = 1)
        tf.summary.scalar('iou_coef', iou, step = 1)
        tf.summary.scalar('dice_coef', dice, step = 1)
        
        # -----------------------------------------------------
        
        # clean memory
        K.clear_session()
        del unetmodel

In [ ]:
#train the model
session_num = 0
for learning_rate in HP_LEARNINGRATE.domain.values:
    for dropout in HP_DROPOUT.domain.values:
        for layer in HP_LAYERS.domain.values:
            run_name = "run-{}".format(session_num)
            print('--- Starting trial: {}'.format(run_name))

            # create hyper-parameter
            hparams = {
                HP_LEARNINGRATE: learning_rate,
                HP_DROPOUT: dropout,
                HP_LAYERS: layer,
            }
            print(hparams)
            print({h.name: hparams[h] for h in hparams})

            # build model and traning
            run(run_name, hparamtuning_dir, hparams)

            session_num += 1


--- Starting trial: run-0
{HParam(name='learning_rate', domain=Discrete([0.0001, 0.001, 0.01, 0.1]), display_name=None, description=None): 0.0001, HParam(name='dropout', domain=Discrete([0.5, 0.7]), display_name=None, description=None): 0.5, HParam(name='layers', domain=Discrete([4, 5]), display_name=None, description=None): 4}
{'learning_rate': 0.0001, 'dropout': 0.5, 'layers': 4}
run-0 does not exist in D:PerlmutterData\logs\fit\nucleus\2020_02_06\2020_02_06_16_39
model does not exist in D:PerlmutterData\logs\fit\nucleus\2020_02_06\2020_02_06_16_39\run-0
learning rate: 0.0001
dropout: 0.5
num layers: 4
Epoch 1/20
372/373 [============================>.] - ETA: 0s - loss: 0.6283 - accuracy: 0.6860 - iou_coef: 0.1170 - dice_coef: 0.1807
Epoch 00001: val_accuracy improved from -inf to 0.54610, saving model to D:PerlmutterData\logs\fit\nucleus\2020_02_06\2020_02_06_16_39\run-0\model\model_2020_02_06_16_39.h5
373/373 [==============================] - 336s 901ms/step - loss: 0.6276 - accu

Epoch 18/20
372/373 [============================>.] - ETA: 0s - loss: 0.1236 - accuracy: 0.9638 - iou_coef: 0.3683 - dice_coef: 0.3918
Epoch 00018: val_accuracy did not improve from 0.95138
373/373 [==============================] - 322s 863ms/step - loss: 0.1235 - accuracy: 0.9639 - iou_coef: 0.3684 - dice_coef: 0.3919 - val_loss: 0.5341 - val_accuracy: 0.8379 - val_iou_coef: 0.3328 - val_dice_coef: 0.3681
Epoch 19/20
372/373 [============================>.] - ETA: 0s - loss: 0.1232 - accuracy: 0.9638 - iou_coef: 0.3671 - dice_coef: 0.3908
Epoch 00019: val_accuracy did not improve from 0.95138

Epoch 00019: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
373/373 [==============================] - 322s 864ms/step - loss: 0.1230 - accuracy: 0.9638 - iou_coef: 0.3669 - dice_coef: 0.3906 - val_loss: 0.7836 - val_accuracy: 0.7750 - val_iou_coef: 0.3018 - val_dice_coef: 0.3348
Epoch 20/20
372/373 [============================>.] - ETA: 0s - loss: 0.1207 - accuracy: 0.964

373/373 [==============================] - 430s 1s/step - loss: 0.1498 - accuracy: 0.9561 - iou_coef: 0.3378 - dice_coef: 0.3647 - val_loss: 0.3996 - val_accuracy: 0.8719 - val_iou_coef: 0.3719 - val_dice_coef: 0.4134
Epoch 15/20
372/373 [============================>.] - ETA: 1s - loss: 0.1437 - accuracy: 0.9579 - iou_coef: 0.3422 - dice_coef: 0.3682
Epoch 00015: val_accuracy did not improve from 0.87195
373/373 [==============================] - 425s 1s/step - loss: 0.1434 - accuracy: 0.9580 - iou_coef: 0.3419 - dice_coef: 0.3680 - val_loss: 0.6765 - val_accuracy: 0.7944 - val_iou_coef: 0.2562 - val_dice_coef: 0.2833
Epoch 16/20
372/373 [============================>.] - ETA: 1s - loss: 0.1375 - accuracy: 0.9599 - iou_coef: 0.3446 - dice_coef: 0.3697
Epoch 00016: val_accuracy did not improve from 0.87195
373/373 [==============================] - 425s 1s/step - loss: 0.1373 - accuracy: 0.9600 - iou_coef: 0.3449 - dice_coef: 0.3699 - val_loss: 0.3523 - val_accuracy: 0.8638 - val_iou_c

373/373 [==============================] - 322s 863ms/step - loss: 0.2003 - accuracy: 0.9343 - iou_coef: 0.3130 - dice_coef: 0.3490 - val_loss: 0.3527 - val_accuracy: 0.8496 - val_iou_coef: 0.3961 - val_dice_coef: 0.4555
Epoch 9/20
372/373 [============================>.] - ETA: 0s - loss: 0.1798 - accuracy: 0.9431 - iou_coef: 0.3219 - dice_coef: 0.3548
Epoch 00009: val_accuracy improved from 0.85220 to 0.86858, saving model to D:PerlmutterData\logs\fit\nucleus\2020_02_06\2020_02_06_16_39\run-2\model\model_2020_02_06_16_39.h5
373/373 [==============================] - 325s 871ms/step - loss: 0.1795 - accuracy: 0.9432 - iou_coef: 0.3220 - dice_coef: 0.3549 - val_loss: 0.3327 - val_accuracy: 0.8686 - val_iou_coef: 0.3653 - val_dice_coef: 0.4174
Epoch 10/20
372/373 [============================>.] - ETA: 0s - loss: 0.1691 - accuracy: 0.9485 - iou_coef: 0.3372 - dice_coef: 0.3681
Epoch 00010: val_accuracy improved from 0.86858 to 0.90801, saving model to D:PerlmutterData\logs\fit\nucleus\2

Epoch 5/20
372/373 [============================>.] - ETA: 1s - loss: 0.5341 - accuracy: 0.7001 - iou_coef: 0.1419 - dice_coef: 0.2062
Epoch 00005: val_accuracy did not improve from 0.55743

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
373/373 [==============================] - 423s 1s/step - loss: 0.5340 - accuracy: 0.7002 - iou_coef: 0.1419 - dice_coef: 0.2062 - val_loss: 0.8794 - val_accuracy: 0.4989 - val_iou_coef: 0.1490 - val_dice_coef: 0.2317
Epoch 6/20
372/373 [============================>.] - ETA: 1s - loss: 0.3322 - accuracy: 0.8567 - iou_coef: 0.2338 - dice_coef: 0.2862
Epoch 00006: val_accuracy improved from 0.55743 to 0.79521, saving model to D:PerlmutterData\logs\fit\nucleus\2020_02_06\2020_02_06_16_39\run-3\model\model_2020_02_06_16_39.h5
373/373 [==============================] - 428s 1s/step - loss: 0.3320 - accuracy: 0.8569 - iou_coef: 0.2339 - dice_coef: 0.2863 - val_loss: 0.7138 - val_accuracy: 0.7952 - val_iou_coef: 0.2336 - val_

...

372/373 [============================>.] - ETA: 1s - loss: 4.7540 - accuracy: 0.6909 - iou_coef: 0.5380 - dice_coef: 0.5380
Epoch 00010: val_accuracy did not improve from 0.57546

Epoch 00010: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
373/373 [==============================] - 426s 1s/step - loss: 4.7503 - accuracy: 0.6911 - iou_coef: 0.5379 - dice_coef: 0.5379 - val_loss: 6.7806 - val_accuracy: 0.5596 - val_iou_coef: 0.4003 - val_dice_coef: 0.4003
Epoch 11/20
372/373 [============================>.] - ETA: 1s - loss: 4.8089 - accuracy: 0.6875 - iou_coef: 0.5402 - dice_coef: 0.5402
Epoch 00011: val_accuracy did not improve from 0.57546
373/373 [==============================] - 426s 1s/step - loss: 4.8053 - accuracy: 0.6877 - iou_coef: 0.5406 - dice_coef: 0.5406 - val_loss: 7.1677 - val_accuracy: 0.5274 - val_iou_coef: 0.4084 - val_dice_coef: 0.4084
Epoch 12/20
372/373 [============================>.] - ETA: 1s - loss: 4.8678 - accuracy: 0.6839 - iou_coef: 0.5

372/373 [============================>.] - ETA: 0s - loss: 10.5626 - accuracy: 0.3102 - iou_coef: 0.3111 - dice_coef: 0.3409
Epoch 00007: val_accuracy did not improve from 0.50533
373/373 [==============================] - 323s 867ms/step - loss: 10.5579 - accuracy: 0.3105 - iou_coef: 0.3114 - dice_coef: 0.3411 - val_loss: 8.3534 - val_accuracy: 0.4544 - val_iou_coef: 0.4552 - val_dice_coef: 0.4875
Epoch 8/20
372/373 [============================>.] - ETA: 0s - loss: 10.5529 - accuracy: 0.3102 - iou_coef: 0.3118 - dice_coef: 0.3424
Epoch 00008: val_accuracy did not improve from 0.50533
373/373 [==============================] - 324s 868ms/step - loss: 10.5536 - accuracy: 0.3102 - iou_coef: 0.3117 - dice_coef: 0.3424 - val_loss: 8.3424 - val_accuracy: 0.4497 - val_iou_coef: 0.4559 - val_dice_coef: 0.4923
Epoch 9/20
372/373 [============================>.] - ETA: 0s - loss: 10.5259 - accuracy: 0.3128 - iou_coef: 0.3135 - dice_coef: 0.3433
Epoch 00009: val_accuracy did not improve from 0.

372/373 [============================>.] - ETA: 1s - loss: 4.7652 - accuracy: 0.6904 - iou_coef: 0.5409 - dice_coef: 0.5409
Epoch 00004: val_accuracy did not improve from 0.55244
373/373 [==============================] - 425s 1s/step - loss: 4.7768 - accuracy: 0.6896 - iou_coef: 0.5404 - dice_coef: 0.5404 - val_loss: 7.0005 - val_accuracy: 0.5454 - val_iou_coef: 0.4064 - val_dice_coef: 0.4064
Epoch 5/20
372/373 [============================>.] - ETA: 1s - loss: 4.8030 - accuracy: 0.6877 - iou_coef: 0.5387 - dice_coef: 0.5387
Epoch 00005: val_accuracy did not improve from 0.55244

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
373/373 [==============================] - 425s 1s/step - loss: 4.8068 - accuracy: 0.6874 - iou_coef: 0.5386 - dice_coef: 0.5386 - val_loss: 7.3160 - val_accuracy: 0.5249 - val_iou_coef: 0.3658 - val_dice_coef: 0.3658
Epoch 6/20
372/373 [============================>.] - ETA: 1s - loss: 4.7045 - accuracy: 0.6946 - iou_coef: 0.5429

373/373 [==============================] - 325s 871ms/step - loss: 4.7485 - accuracy: 0.6902 - iou_coef: 0.5370 - dice_coef: 0.5373 - val_loss: 7.2081 - val_accuracy: 0.5377 - val_iou_coef: 0.3651 - val_dice_coef: 0.3651
Epoch 2/20
372/373 [============================>.] - ETA: 0s - loss: 4.8505 - accuracy: 0.6852 - iou_coef: 0.5352 - dice_coef: 0.5352
Epoch 00002: val_accuracy improved from 0.53772 to 0.53780, saving model to D:PerlmutterData\logs\fit\nucleus\2020_02_06\2020_02_06_16_39\run-10\model\model_2020_02_06_16_39.h5
373/373 [==============================] - 325s 871ms/step - loss: 4.8473 - accuracy: 0.6854 - iou_coef: 0.5351 - dice_coef: 0.5351 - val_loss: 7.1180 - val_accuracy: 0.5378 - val_iou_coef: 0.3974 - val_dice_coef: 0.3974
Epoch 3/20
372/373 [============================>.] - ETA: 0s - loss: 4.7602 - accuracy: 0.6907 - iou_coef: 0.5442 - dice_coef: 0.5442
Epoch 00003: val_accuracy improved from 0.53780 to 0.55546, saving model to D:PerlmutterData\logs\fit\nucleus\2

Epoch 20/20
372/373 [============================>.] - ETA: 0s - loss: 4.7595 - accuracy: 0.6907 - iou_coef: 0.5381 - dice_coef: 0.5381
Epoch 00020: val_accuracy did not improve from 0.56132
50/50 [==============================] - 16s 311ms/step - loss: 7.1784 - accuracy: 0.5343 - iou_coef: 0.3901 - dice_coef: 0.3901
--- Starting trial: run-11
{HParam(name='learning_rate', domain=Discrete([0.0001, 0.001, 0.01, 0.1]), display_name=None, description=None): 0.01, HParam(name='dropout', domain=Discrete([0.5, 0.7]), display_name=None, description=None): 0.7, HParam(name='layers', domain=Discrete([4, 5]), display_name=None, description=None): 5}
{'learning_rate': 0.01, 'dropout': 0.7, 'layers': 5}
run-11 does not exist in D:PerlmutterData\logs\fit\nucleus\2020_02_06\2020_02_06_16_39
model does not exist in D:PerlmutterData\logs\fit\nucleus\2020_02_06\2020_02_06_16_39\run-11
learning rate: 0.01
dropout: 0.7
num layers: 5
Epoch 1/20
372/373 [============================>.] - ETA: 1s - loss: 1

In [ ]:
print("Training finished")